In [1]:
def proper_round(num, dec=0):
    num = str(num)[:str(num).index('.')+dec+2]
    if num[-1]>='5':
      a = num[:-2-(not dec)]       # integer part
      b = int(num[-2-(not dec)])+1 # decimal part
      return float(a)+b**(-dec+1) if a and b == 10 else float(a+str(b))
    return float(num[:-1])

In [2]:
import pandas as pd
import plotly.express as px
import numpy as np
import ipywidgets as widgets
from ipywidgets import interact

In [3]:
df = pd.read_csv('tickets.csv',parse_dates=['creation_date', 'view_date', 'action_date'])
df.head()

,ticket_type,creation_date,view_date,action_date,customer_satisfaction,customer_problem_resolved,user_actioned,user_team
0,live_chat,2021-07-02 08:36:00,2021-07-02 08:36:00,2021-07-02 08:50:00,1,False,291950,orange
1,web_form,2021-07-02 02:37:00,2021-07-02 08:51:00,2021-07-02 09:12:00,1,False,291950,orange
2,web_form,2021-07-02 02:50:00,2021-07-02 09:15:00,2021-07-02 09:38:00,1,False,291950,orange
3,web_form,2021-07-02 02:01:00,2021-07-02 09:44:00,2021-07-02 10:06:00,1,False,291950,orange
4,web_form,2021-07-02 02:41:00,2021-07-02 10:14:00,2021-07-02 10:37:00,2,False,291950,orange


In [4]:
#create dictionaries
color_dict = {1:'#d7191c', 2:'#fdae61', 3:'#ffffbf', 4:'#abd9e9', 5:'#2c7bb6' }
satisfac_dict = {1:'very dissatisfied', 2:'dissatisfied', 3:'neutral', 4:'satisfied', 5:'very satisfied'}

#add columns to dataframe
df['duration'] = df['action_date'] - df['creation_date']
df['color'] = df['customer_satisfaction'].apply(lambda x: color_dict[x])
df['satisfaction'] = df['customer_satisfaction'].apply(lambda x: satisfac_dict[x])
df['user_team'] = df['user_team'].apply(lambda x: 'Team ' + x)

#compute minimum response time
df_meanresponse = df.groupby('user_team')['duration']
mean_response = df_meanresponse.apply(lambda x: np.mean(x)).reset_index()

#create dictionary for each team's response time
responsedict ={}
responsedict['Team blue'] = str(mean_response.iloc[0]['duration'].days) +' days'
responsedict['Team green'] = str(int(proper_round(mean_response.iloc[1]['duration']/pd.Timedelta(hours=1), 0))) + ' hours'
responsedict['Team orange'] = str(mean_response.iloc[2]['duration'].days) +' days'
responsedict['Team red'] = str(int(proper_round(mean_response.iloc[3]['duration']/pd.Timedelta(hours=1), 0))) + ' hours'

#creating new dataframes for the first plotly treemap: customer satisfaction per team  
df1_team = df.groupby(['user_team','customer_satisfaction','color','satisfaction']).count().reset_index().rename(columns={'ticket_type':'number'})
df2_team = df1_team[['user_team','customer_satisfaction','number','color','satisfaction']]
df3_team = df2_team.copy()
df3_team['all'] = 'all'
df3_team['user_team'] = df3_team['user_team'].apply(lambda x: x +', average response time:' + responsedict[x] )

#creating new dataframes for the second plotly treemap: problem resolved/not resolved per team
df_team_resolve = df.groupby(['user_team','customer_problem_resolved']).count().reset_index().rename(columns={'ticket_type':'number'})
df_team_resolve = df_team_resolve[['user_team','customer_problem_resolved','number']]
df_final = df_team_resolve.copy()
df_final['resolve'] = df_final['customer_problem_resolved'].apply(lambda x: 'Ticket is resolved' if x else 'Ticket is not resolved')
df_final['all'] = 'all'
df_final['user_team'] = df_final['user_team'].apply(lambda x: x +', average response time:' + responsedict[x] )

#creating new dataframes for the third plotly treemap: customer satisfaction per ticket type
df_meanresponse_type = df.groupby('ticket_type')['duration']
mean_response_type = df_meanresponse_type.apply(lambda x: np.mean(x)).reset_index()

responsedict_type ={}
responsedict_type['email'] = str(mean_response_type.iloc[0]['duration'].days) +' days'
responsedict_type['live_chat'] = str(int(proper_round(mean_response_type.iloc[1]['duration']/pd.Timedelta(minutes=1), 0))) + ' minutes'
responsedict_type['web_form'] = str(mean_response_type.iloc[2]['duration'].days) +' days'

df1_ticket = df.groupby(['ticket_type','customer_satisfaction','color','satisfaction']).count().reset_index().rename(columns={'user_team':'number'})
df2_ticket = df1_ticket[['ticket_type','customer_satisfaction','number','color','satisfaction']]
df3_ticket = df2_ticket.copy()
df3_ticket['all'] = 'all'
df3_ticket['ticket_type'] = df3_ticket['ticket_type'].apply(lambda x: x +': ' + responsedict_type[x] )

#creating new dataframes for the fourth plotly treemap: problem resolved/not resolved per ticket type
df1_type_resolve = df.groupby(['ticket_type','customer_problem_resolved']).count().reset_index().rename(columns={'user_team':'number'})
df2_type_resolve = df1_type_resolve[['ticket_type','customer_problem_resolved','number']]
df3_type_resolve = df2_type_resolve.copy()
df3_type_resolve['resolve'] = df3_type_resolve['customer_problem_resolved'].apply(lambda x: 'Ticket is resolved' if x else 'Ticket is not resolved')
df3_type_resolve['all'] = 'all'
df3_type_resolve['ticket_type'] = df3_type_resolve['ticket_type'].apply(lambda x: x +': ' + responsedict_type[x] )

In [5]:
option={'team\'s satisfaction':1,
        'team\'s problem solving':2,
        'ticket type\'s satisfaction':3,
       'ticket type\'s problem solving':4}
@interact(choice = widgets.RadioButtons(
    options = option,
    description='explore:',
    disabled=False
))
def f(choice):
    if choice == 1:
        fig = px.treemap(df3_team, path=['all', 'user_team', 'satisfaction'], values='number', color='satisfaction'
      ,color_discrete_map={'(?)':'black','very dissatisfied':'#d7191c','dissatisfied':'#fdae61','neutral':'#ffffbf','satisfied':'#abd9e9','very satisfied':'#2c7bb6'} )
        fig.update_traces(root_color="lightgrey")
        fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
        fig.data[0].hovertemplate = 'Number of Tickets: %{value}'

        cm = {
               "Team red, average response time:11 hours": "red",
               "Team green, average response time:14 hours": "green",
                "Team orange, average response time:7 days": "orange",
                "Team blue, average response time:3 days": "blue"
               }
        fig.for_each_trace(
            lambda t: t.update(
            marker_colors=[
            cm[id.split("/")[1]] if len(id.split("/")) == 2 else c
            for c, id in zip(t.marker.colors, t.ids)
                 ]
                 )
                 )
        fig.show()
    
    if choice == 2:
        fig = px.treemap(df_final, path=['all', 'user_team', 'resolve'], values='number', color='resolve'
      ,color_discrete_map={'(?)':'black','Ticket is not resolved':'#bdbdbd','Ticket is resolved':'#a1d99b'} )
        fig.update_traces(root_color="lightgrey")
        fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
        fig.data[0].hovertemplate = 'Number of Tickets: %{value}'

        cm = {
              "Team red, average response time:11 hours": "red",
              "Team green, average response time:14 hours": "green",
               "Team orange, average response time:7 days": "orange",
               "Team blue, average response time:3 days": "blue"
                }
        fig.for_each_trace(
             lambda t: t.update(
              marker_colors=[
              cm[id.split("/")[1]] if len(id.split("/")) == 2 else c
               for c, id in zip(t.marker.colors, t.ids)
               ]
               )
               )
        fig.show()
    
    if choice == 3:
            fig = px.treemap(df3_ticket, path=['all', 'ticket_type', 'satisfaction'], values='number', color='satisfaction'
         ,color_discrete_map={'(?)':'black','very dissatisfied':'#d7191c','dissatisfied':'#fdae61','neutral':'#ffffbf','satisfied':'#abd9e9','very satisfied':'#2c7bb6'} )
            fig.update_traces(root_color="lightgrey")
            fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
            fig.data[0].hovertemplate = 'Number of Tickets: %{value}'
            fig.show()
    if choice == 4:    
            fig = px.treemap(df3_type_resolve, path=['all', 'ticket_type', 'resolve'], values='number', color='resolve'
            ,color_discrete_map={'(?)':'black','Ticket is not resolved':'#bdbdbd','Ticket is resolved':'#a1d99b'} )
            fig.update_traces(root_color="lightgrey")
            fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
            fig.data[0].hovertemplate = 'Number of Tickets: %{value}'
            fig.show()

interactive(children=(RadioButtons(description='explore:', options={"team's satisfaction": 1, "team's problem …